In [24]:
from py2neo import Graph
import pandas as pd

pd.set_option('display.max_colwidth', -1)

In [2]:
graph = Graph("bolt://localhost", auth=("neo4j", "neo"))

In [11]:
popular_users_query = """
MATCH (u:User)
RETURN u.name AS user, size((u)-[:WROTE]->()) AS reviews
ORDER BY reviews DESC
LIMIT 10
"""

graph.run(popular_users_query).to_data_frame()

,reviews,user
0,28,Julie
1,25,Teresa
2,25,Casey
3,23,Alex
4,16,Rich
5,15,Gail
6,15,Christopher
7,14,Mark
8,13,Aubrey
9,13,Geoffrey


In [18]:
collabarative_filtering_query = """
MATCH (u:User {id: $userId})-[:WROTE]->()-[:REVIEWS]->(listing:Listing),
      (other)-[:WROTE]->()-[:REVIEWS]->(listing)
WHERE u <> other      
WITH other, count(distinct listing) AS commonListings      
RETURN other, commonListings
ORDER BY commonListings DESC
LIMIT 10
"""

user_id = "2011754"

graph.run(collabarative_filtering_query, {"userId": user_id}).to_data_frame()

,commonListings,other
0,4,"{'name': 'Casey', 'id': '4363395'}"
1,4,"{'name': 'Teresa', 'id': '5256115'}"
2,3,"{'name': 'Lisa', 'id': '587934'}"
3,3,"{'name': 'Meghan', 'id': '5759317'}"
4,3,"{'name': 'John', 'id': '9953891'}"
5,2,"{'name': 'Jennice', 'id': '73211'}"
6,2,"{'name': 'Maggie', 'id': '20110260'}"
7,2,"{'name': 'Roman', 'id': '3359720'}"
8,2,"{'name': 'Echoin', 'id': '1809936'}"
9,2,"{'name': 'Alexandra', 'id': '941328'}"


In [27]:
collabarative_filtering_query = """
MATCH (u:User {id: $userId})-[:WROTE]->()-[:REVIEWS]->(listing:Listing),
      (other)-[:WROTE]->()-[:REVIEWS]->(listing)
WHERE u <> other      
WITH u, other, count(distinct listing) AS commonListings
LIMIT 10
MATCH (other)-[:WROTE]->()-[:REVIEWS]->(listing)
WHERE not((u)-[:WROTE]->()-[:REVIEWS]->(listing))
RETURN listing, count(*) AS times
"""

user_id = "2011754"
graph.run(collabarative_filtering_query, {"userId": user_id}).to_data_frame()

,listing,times
0,"{'bedrooms': 2, 'availability365': 363, 'price': 250.0, 'accommodates': 4, 'propertyType': 'House', 'name': 'South Central Treetop Guesthouse!!!', 'cleaningFee': 35.0, 'id': '714922', 'bathrooms': 1}",1
